In [1]:
import pprint
from os import environ

import pyalex
import pandas as pd
from sqlalchemy import create_engine


In [2]:
# setup
pyalex.config.email = environ.get("OPENALEX_CONTACT_EMAIL")
# db
db_url = f"postgresql+psycopg://{environ.get('DB_USER')}:{environ.get('DB_PASSWORD')}@{environ.get('DB_HOST')}:5432/{environ.get('DB_NAME')}"
engine = create_engine(db_url)

In [3]:
# function for fetching openalex entities, returning dataframe
def fetch_all_objects(entity_type: pyalex.api.BaseOpenAlex, fields: list[str]) -> pd.DataFrame:
    from itertools import chain
    results = []
    for result in chain(*entity_type.select(fields).sort(id="asc").paginate(per_page=200)):
        results.append(result)
    return pd.DataFrame(results)

In [4]:
domains = fetch_all_objects(pyalex.Domains(), ["id", "display_name", "description", "ids", "updated_date"])
fields = fetch_all_objects(pyalex.Fields(), ["id", "display_name", "description", "ids", "updated_date", "domain"])
subfields = fetch_all_objects(pyalex.Subfields(), ["id", "display_name", "description", "ids", "updated_date", "field"])
topics = fetch_all_objects(pyalex.Topics(), ["id", "display_name", "description", "ids", "updated_date", "subfield", "keywords"])

In [5]:
topics

,id,display_name,description,ids,updated_date,subfield,keywords
0,https://openalex.org/T10001,Tectonic and Geochronological Evolution of Oro...,This cluster of papers focuses on the tectonic...,"{'openalex': 'https://openalex.org/T10001', 'w...",2024-03-04T05:00:09.665798,"{'id': 'https://openalex.org/subfields/1908', ...","[Zircon, Geochronology, Tectonics, Granitic Ro..."
1,https://openalex.org/T10002,Advancements in Density Functional Theory,This cluster of papers represents advancements...,"{'openalex': 'https://openalex.org/T10002', 'w...",2024-03-04T05:03:43.042245,"{'id': 'https://openalex.org/subfields/3107', ...","[Density Functional Theory, Dispersion Correct..."
2,https://openalex.org/T10003,Knowledge Management and Organizational Innova...,This cluster of papers revolves around the top...,"{'openalex': 'https://openalex.org/T10003', 'w...",2024-03-04T05:01:52.245888,"{'id': 'https://openalex.org/subfields/1408', ...","[Dynamic Capabilities, Knowledge Transfer, Bus..."
3,https://openalex.org/T10004,Soil Carbon Dynamics and Nutrient Cycling in E...,This cluster of papers explores the dynamics o...,"{'openalex': 'https://openalex.org/T10004', 'w...",2024-03-04T05:10:09.950055,"{'id': 'https://openalex.org/subfields/1111', ...","[Soil Carbon Sequestration, Nitrogen Cycle, Mi..."
4,https://openalex.org/T10005,Biodiversity Conservation and Ecosystem Manage...,This cluster of papers focuses on the conserva...,"{'openalex': 'https://openalex.org/T10005', 'w...",2024-03-04T05:10:06.251662,"{'id': 'https://openalex.org/subfields/2309', ...","[Biodiversity, Conservation, Ecosystem, Invasi..."
...,...,...,...,...,...,...,...
4511,https://openalex.org/T14517,History of Science and Knowledge Production,This cluster of papers covers a wide range of ...,"{'openalex': 'https://openalex.org/T14517', 'w...",2024-03-04T05:09:43.370004,"{'id': 'https://openalex.org/subfields/1207', ...","[History, Science, Knowledge Production, Medic..."
4512,https://openalex.org/T14518,Baseball's Influence on American Culture and S...,This cluster of papers explores the profound i...,"{'openalex': 'https://openalex.org/T14518', 'w...",2024-03-04T05:07:34.043463,"{'id': 'https://openalex.org/subfields/1202', ...","[Baseball, American Culture, Societal Influenc..."
4513,https://openalex.org/T14519,Digital Education and Knowledge Economy,This cluster of papers explores the intersecti...,"{'openalex': 'https://openalex.org/T14519', 'w...",2024-03-04T05:12:22.422837,"{'id': 'https://openalex.org/subfields/1710', ...","[Digital Education, E-Learning, Knowledge Econ..."
4514,https://openalex.org/T14520,Slow Cities Movement and Sustainable Urban Dev...,This cluster of papers explores the Slow Citie...,"{'openalex': 'https://openalex.org/T14520', 'w...",2024-03-04T05:00:34.355241,"{'id': 'https://openalex.org/subfields/3322', ...","[Slow Cities, Sustainable Development, Urban S..."


In [6]:
# function for preprocessing dataframe for embedding
def preprocess(df: pd.DataFrame, type_name: str) -> pd.DataFrame:
    df = df.copy()
    # id needs to be extracted from id column, which contains openalex urls
    id_regex = r"(\d+)$" # id is the number at the end of the url
    df["id"] = df["id"].str.extract(id_regex)
    df.set_index("id", verify_integrity=True, inplace=True)
    # create separate columns for wikidata and wikipedia urls from ids column
    df["wikipedia"] = df["ids"].apply(lambda x: x.get("wikipedia", None))
    # topics don't have wikidata urls
    if type_name != "topic":
        df["wikidata"] = df["ids"].apply(lambda x: x.get("wikidata", None))
    df.drop("ids", axis=1, inplace=True)
    
    
    type_to_parent_type = {
        "field": "domain",
        "subfield": "field",
        "topic": "subfield"
    }
    # extract parent id
    if type_name != "domain": # domains have no parent
        # extract parent url from dict
        df[type_to_parent_type[type_name]] = df[type_to_parent_type[type_name]].apply(lambda x: x.get("id", None))
        # and extract the id
        df[type_to_parent_type[type_name]] = df[type_to_parent_type[type_name]].str.extract(id_regex)
    
    return df

In [7]:
preprocessed_domains = preprocess(domains, "domain")
preprocessed_fields = preprocess(fields, "field")
preprocessed_subfields = preprocess(subfields, "subfield")
preprocessed_topics = preprocess(topics, "topic")

In [8]:
preprocessed_topics

,display_name,description,updated_date,subfield,keywords,wikipedia
id,,,,,,
10001,Tectonic and Geochronological Evolution of Oro...,This cluster of papers focuses on the tectonic...,2024-03-04T05:00:09.665798,1908,"[Zircon, Geochronology, Tectonics, Granitic Ro...",https://en.wikipedia.org/wiki/Geochronology
10002,Advancements in Density Functional Theory,This cluster of papers represents advancements...,2024-03-04T05:03:43.042245,3107,"[Density Functional Theory, Dispersion Correct...",https://en.wikipedia.org/wiki/Density_function...
10003,Knowledge Management and Organizational Innova...,This cluster of papers revolves around the top...,2024-03-04T05:01:52.245888,1408,"[Dynamic Capabilities, Knowledge Transfer, Bus...",https://en.wikipedia.org/wiki/Knowledge_manage...
10004,Soil Carbon Dynamics and Nutrient Cycling in E...,This cluster of papers explores the dynamics o...,2024-03-04T05:10:09.950055,1111,"[Soil Carbon Sequestration, Nitrogen Cycle, Mi...",https://en.wikipedia.org/wiki/Soil_carbon_dyna...
10005,Biodiversity Conservation and Ecosystem Manage...,This cluster of papers focuses on the conserva...,2024-03-04T05:10:06.251662,2309,"[Biodiversity, Conservation, Ecosystem, Invasi...",https://en.wikipedia.org/wiki/Biodiversity_con...
...,...,...,...,...,...,...
14517,History of Science and Knowledge Production,This cluster of papers covers a wide range of ...,2024-03-04T05:09:43.370004,1207,"[History, Science, Knowledge Production, Medic...",https://en.wikipedia.org/wiki/History_of_science
14518,Baseball's Influence on American Culture and S...,This cluster of papers explores the profound i...,2024-03-04T05:07:34.043463,1202,"[Baseball, American Culture, Societal Influenc...",https://en.wikipedia.org/wiki/Baseball_in_the_...
14519,Digital Education and Knowledge Economy,This cluster of papers explores the intersecti...,2024-03-04T05:12:22.422837,1710,"[Digital Education, E-Learning, Knowledge Econ...",https://en.wikipedia.org/wiki/Digital_education


In [9]:
# function to generate the string representation for embedding
def generate_string_representation_for_embedding(df: pd.DataFrame, column_prefix_map: dict) -> pd.Series:
    # first, check if all columns are present in the dataframe
    for column in column_prefix_map.keys():
        if column not in df.columns:
            raise ValueError(f"Column {column} not found in the dataframe.")
    
    # Initialize a list to store the concatenated strings for each row
    concatenated_strings = pd.Series(dtype=str)
    
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Initialize an empty list to store the strings for the current row
        row_strings = []
        # Iterate over the column map dictionary
        for column_name, prefix in column_prefix_map.items():
            column_value = row[column_name]
            if type(column_value) == list:
                column_value = ", ".join(column_value)
            # Concatenate the column's value for the current row with the corresponding string
            row_strings.append(prefix + ": " + str(column_value).strip())
        # Join the strings for the current row with "; " and add to the list
        concatenated_strings.at[index] ="; ".join(row_strings)
        
    return concatenated_strings

In [10]:
preprocessed_domains["string_representation"] = generate_string_representation_for_embedding(preprocessed_domains, {
    "display_name": "Domain",
    "description": "Description"
})
preprocessed_fields["string_representation"] = generate_string_representation_for_embedding(preprocessed_fields, {
    "display_name": "Field",
    "description": "Description"
})
preprocessed_subfields["string_representation"] = generate_string_representation_for_embedding(preprocessed_subfields, {
    "display_name": "Subfield",
    "description": "Description"
})
preprocessed_topics["string_representation"] = generate_string_representation_for_embedding(preprocessed_topics, {
    "display_name": "Topic",
    "description": "Description",
    "keywords": "Keywords"
})

In [11]:
preprocessed_fields

,display_name,description,updated_date,domain,wikipedia,wikidata,string_representation
id,,,,,,,
11,Agricultural and Biological Sciences,"study of plant and animal biology for food, fu...",2024-03-04T05:00:45.631375,1,https://en.wikipedia.org/wiki/Agriculture,https://www.wikidata.org/wiki/Q173113,Field: Agricultural and Biological Sciences; D...
12,Arts and Humanities,"examination of human culture, including art, l...",2024-03-04T05:00:47.362383,2,https://en.wikipedia.org/wiki/Humanities,https://www.wikidata.org/wiki/Q80083,Field: Arts and Humanities; Description: exami...
13,"Biochemistry, Genetics and Molecular Biology",study of chemical processes in living organisms,2024-03-04T05:00:45.092489,1,https://en.wikipedia.org/wiki/Biochemistry,https://www.wikidata.org/wiki/Q7094,"Field: Biochemistry, Genetics and Molecular Bi..."
14,"Business, Management and Accounting",administration of a commercial enterprise,2024-03-04T05:00:46.514408,2,https://en.wikipedia.org/wiki/Business_adminis...,https://www.wikidata.org/wiki/Q18564125,"Field: Business, Management and Accounting; De..."
15,Chemical Engineering,branch of science that applies physical scienc...,2024-03-04T05:00:46.353442,3,https://en.wikipedia.org/wiki/Chemical_enginee...,https://www.wikidata.org/wiki/Q83588,Field: Chemical Engineering; Description: bran...
16,Chemistry,branch of physical science concerned with the ...,2024-03-04T05:00:45.782490,3,https://en.wikipedia.org/wiki/Chemistry,https://www.wikidata.org/wiki/Q2329,Field: Chemistry; Description: branch of physi...
17,Computer Science,study of computation,2024-03-04T05:00:46.682573,3,https://en.wikipedia.org/wiki/Computer_science,https://www.wikidata.org/wiki/Q21198,Field: Computer Science; Description: study of...
18,Decision Sciences,branch of applied probability theory,2024-03-04T05:00:47.544354,2,https://en.wikipedia.org/wiki/Decision_theory,https://www.wikidata.org/wiki/Q177571,Field: Decision Sciences; Description: branch ...
19,Earth and Planetary Sciences,all fields of natural science related to the p...,2024-03-04T05:00:53.474931,3,https://en.wikipedia.org/wiki/Earth_science,https://www.wikidata.org/wiki/Q107287417,Field: Earth and Planetary Sciences; Descripti...


In [12]:
# function to calculate sum of tokens for a Series
import tiktoken
def calculate_total_tokens(series: pd.Series, encoding_name: str = "cl100k_base") -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    return series.apply(lambda x: len(encoding.encode(x))).sum()
# function to calculate embedding costs for a count of tokens
def calculate_embedding_costs(num_tokens: int, price_per_m_tokens: float = 0.13) -> float:
    price_per_token = price_per_m_tokens / 1000000
    return num_tokens * price_per_token
# TODO: function to calculate embedding costs for a Series

In [13]:
total_tokens_for_domains = calculate_total_tokens(preprocessed_domains["string_representation"])
total_tokens_for_fields = calculate_total_tokens(preprocessed_fields["string_representation"])
total_tokens_for_subfields = calculate_total_tokens(preprocessed_subfields["string_representation"])
total_tokens_for_topics = calculate_total_tokens(preprocessed_topics["string_representation"])

embedding_costs_for_domains = calculate_embedding_costs(total_tokens_for_domains)
embedding_costs_for_fields = calculate_embedding_costs(total_tokens_for_fields)
embedding_costs_for_subfields = calculate_embedding_costs(total_tokens_for_subfields)
embedding_costs_for_topics = calculate_embedding_costs(total_tokens_for_topics)

print(f"Total tokens for Domains: {total_tokens_for_domains} - Embedding costs: {embedding_costs_for_domains}")
print(f"Total tokens for Fields: {total_tokens_for_fields} - Embedding costs: {embedding_costs_for_fields}")
print(f"Total tokens for Subfields: {total_tokens_for_subfields} - Embedding costs: {embedding_costs_for_subfields}")
print(f"Total tokens for Topics: {total_tokens_for_topics} - Embedding costs: {embedding_costs_for_topics}")
print(f"Total embedding costs: {embedding_costs_for_domains + embedding_costs_for_fields + embedding_costs_for_subfields + embedding_costs_for_topics}")

Total tokens for Domains: 92 - Embedding costs: 1.196e-05
Total tokens for Fields: 559 - Embedding costs: 7.267e-05
Total tokens for Subfields: 5092 - Embedding costs: 0.00066196
Total tokens for Topics: 550122 - Embedding costs: 0.07151586
Total embedding costs: 0.07226245


In [14]:
from openai import OpenAI
# function to get embeddings via OpenAI-API. Max. input is 2048 strings
def get_embeddings(input: pd.Series, batch_size: int = 2048, model="text-embedding-3-large", dimensions=1024) -> pd.Series:
    client = OpenAI() # requires OPENAI_API_KEY environment variable to be set
    # batch requests to speed up API calls
    embeddings = []
    for i in range(0, len(input), batch_size):
        batch = input.iloc[i:i+batch_size]
        response = client.embeddings.create(input=batch.tolist(), model=model, dimensions=dimensions)
        embeddings.extend([r.embedding for r in response.data])
    return pd.Series(embeddings, index=input.index)

def get_embedding(text: str, model="text-embedding-3-large", dimensions=1024):
    client = OpenAI() # requires OPENAI_API_KEY environment variable to be set
    response = client.embeddings.create(input=[text], model=model, dimensions=dimensions)
    return response.data[0].embedding

In [15]:
preprocessed_domains["embedding"] = get_embeddings(preprocessed_domains["string_representation"])
preprocessed_fields["embedding"] = get_embeddings(preprocessed_fields["string_representation"])
preprocessed_subfields["embedding"] = get_embeddings(preprocessed_subfields["string_representation"])
preprocessed_topics["embedding"] = get_embeddings(preprocessed_topics["string_representation"])

In [79]:
preprocessed_fields

,display_name,description,updated_date,domain,wikidata,wikipedia,string_representation,embedding
id,,,,,,,,
11,Agricultural and Biological Sciences,"study of plant and animal biology for food, fu...",2024-03-04T05:00:45.631375,1,https://www.wikidata.org/wiki/Q173113,https://en.wikipedia.org/wiki/Agriculture,Field: Agricultural and Biological Sciences; D...,"[0.01420079730451107, -0.018364083021879196, -..."
12,Arts and Humanities,"examination of human culture, including art, l...",2024-03-04T05:00:47.362383,2,https://www.wikidata.org/wiki/Q80083,https://en.wikipedia.org/wiki/Humanities,Field: Arts and Humanities; Description: exami...,"[-0.035589590668678284, 0.014225154183804989, ..."
13,"Biochemistry, Genetics and Molecular Biology",study of chemical processes in living organisms,2024-03-04T05:00:45.092489,1,https://www.wikidata.org/wiki/Q7094,https://en.wikipedia.org/wiki/Biochemistry,"Field: Biochemistry, Genetics and Molecular Bi...","[0.006245376076549292, -0.0198944304138422, -0..."
14,"Business, Management and Accounting",administration of a commercial enterprise,2024-03-04T05:00:46.514408,2,https://www.wikidata.org/wiki/Q18564125,https://en.wikipedia.org/wiki/Business_adminis...,"Field: Business, Management and Accounting; De...","[0.021300891414284706, 0.03298407420516014, 3...."
15,Chemical Engineering,branch of science that applies physical scienc...,2024-03-04T05:00:46.353442,3,https://www.wikidata.org/wiki/Q83588,https://en.wikipedia.org/wiki/Chemical_enginee...,Field: Chemical Engineering; Description: bran...,"[-0.04571014270186424, -0.035771507769823074, ..."
16,Chemistry,branch of physical science concerned with the ...,2024-03-04T05:00:45.782490,3,https://www.wikidata.org/wiki/Q2329,https://en.wikipedia.org/wiki/Chemistry,Field: Chemistry; Description: branch of physi...,"[-0.0015265783295035362, -0.021598078310489655..."
17,Computer Science,study of computation,2024-03-04T05:00:46.682573,3,https://www.wikidata.org/wiki/Q21198,https://en.wikipedia.org/wiki/Computer_science,Field: Computer Science; Description: study of...,"[0.00019407148647587746, -0.011263850145041943..."
18,Decision Sciences,branch of applied probability theory,2024-03-04T05:00:47.544354,2,https://www.wikidata.org/wiki/Q177571,https://en.wikipedia.org/wiki/Decision_theory,Field: Decision Sciences; Description: branch ...,"[-0.016336772590875626, 0.0019115452887490392,..."
19,Earth and Planetary Sciences,all fields of natural science related to the p...,2024-03-04T05:00:53.474931,3,https://www.wikidata.org/wiki/Q107287417,https://en.wikipedia.org/wiki/Earth_science,Field: Earth and Planetary Sciences; Descripti...,"[-0.009837082587182522, -0.002517000073567033,..."


In [18]:
# function to prepare dataframe for database insertion
def prepare_dataframe_for_db(df: pd.DataFrame) -> pd.DataFrame:
    df_for_db = df.copy()
    df_for_db.drop(["string_representation"], axis=1, inplace=True)
    df_for_db.rename(columns={"display_name": "name", "domain": "domain_id", "field": "field_id", "subfield": "subfield_id"}, inplace=True)
    return df_for_db

In [19]:
domains_for_db = prepare_dataframe_for_db(preprocessed_domains)
fields_for_db = prepare_dataframe_for_db(preprocessed_fields)
subfields_for_db = prepare_dataframe_for_db(preprocessed_subfields)
topics_for_db = prepare_dataframe_for_db(preprocessed_topics)

In [23]:
from sqlalchemy import String, DateTime, Integer, ARRAY
from pgvector.sqlalchemy import Vector

# function to insert dataframe into database
def insert_dataframe_into_db(df: pd.DataFrame, type_name: str):
    type_to_parent_type = {
        "field": "domain",
        "subfield": "field",
        "topic": "subfield"
    }
    
    column_name_to_column_type = {
        "id": Integer,
        "name": String,
        'description': String,
        "updated_date": DateTime,
        "wikipedia": String,
        "embedding": Vector(1024)
    }
    
    if type_name != "topic":
        column_name_to_column_type["wikidata"] = String
    else:
        column_name_to_column_type["keywords"] = ARRAY(String, dimensions=1)
        
    if type_name != "domain":
        column_name_to_column_type[type_to_parent_type[type_name] + "_id"] = Integer
    
    table_name = f"openalex_{type_name}"
    
    
    df.to_sql(table_name, con=engine, if_exists="append", dtype=column_name_to_column_type)

In [26]:
insert_dataframe_into_db(domains_for_db, "domain")
insert_dataframe_into_db(fields_for_db, "field")
insert_dataframe_into_db(subfields_for_db, "subfield")
insert_dataframe_into_db(topics_for_db, "topic")

In [59]:
# get embeddings for a test query
query = """
Multi-head attention allows the model to jointly attend to information from different representation
subspaces at different positions. With a single attention head, averaging inhibits this.
MultiHead(Q, K, V ) = Concat(head1, ..., headh)W O
where headi = Attention(QW Q
i , KW K
i , V W V
i )
Where the projections are parameter matrices W Q
i ∈ Rdmodel×dk , W K
i ∈ Rdmodel×dk , W V
i ∈ Rdmodel×dv
and W O ∈ Rhdv ×dmodel .
In this work we employ h = 8 parallel attention layers, or heads. For each of these we use
dk = dv = dmodel/h = 64. Due to the reduced dimension of each head, the total computational cost
is similar to that of single-head attention with full dimensionality.
3.2.3 Applications of Attention in our Model
The Transformer uses multi-head attention in three different ways:
• In "encoder-decoder attention" layers, the queries come from the previous decoder layer,
and the memory keys and values come from the output of the encoder. This allows every
position in the decoder to attend over all positions in the input sequence. This mimics the
typical encoder-decoder attention mechanisms in sequence-to-sequence models such as
[38, 2, 9].
• The encoder contains self-attention layers. In a self-attention layer all of the keys, values
and queries come from the same place, in this case, the output of the previous layer in the
encoder. Each position in the encoder can attend to all positions in the previous layer of the
encoder.
• Similarly, self-attention layers in the decoder allow each position in the decoder to attend to
all positions in the decoder up to and including that position. We need to prevent leftward
information ﬂow in the decoder to preserve the auto-regressive property. We implement this
inside of scaled dot-product attention by masking out (setting to −∞) all values in the input
of the softmax which correspond to illegal connections. See Figure 2.
3.3 Position-wise Feed-Forward Networks
In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully
connected feed-forward network, which is applied to each position separately and identically. This
consists of two linear transformations with a ReLU activation in between.
FFN(x) = max(0, xW1 + b1)W2 + b2 (2)
While the linear transformations are the same across different positions, they use different parameters
from layer to layer. Another way of describing this is as two convolutions with kernel size 1.
The dimensionality of input and output is dmodel = 512, and the inner-layer has dimensionality
df f = 2048.
3.4 Embeddings and Softmax
Similarly to other sequence transduction models, we use learned embeddings to convert the input
tokens and output tokens to vectors of dimension dmodel. We also use the usual learned linear transfor-
mation and softmax function to convert the decoder output to predicted next-token probabilities. In
our model, we share the same weight matrix between the two embedding layers and the pre-softmax
linear transformation, similar to [ 30 ]. In the embedding layers, we multiply those weights by √dmodel.
3.5 Positional Encoding
Since our model contains no recurrence and no convolution, in order for the model to make use of the
order of the sequence, we must inject some information about the relative or absolute position of the
5
Table 1: Maximum path lengths, per-layer complexity and minimum number of sequential operations
for different layer types. n is the sequence length, d is the representation dimension, k is the kernel
size of convolutions and r the size of the neighborhood in restricted self-attention.
Layer Type Complexity per Layer Sequential Maximum Path Length
Operations
Self-Attention O(n2 · d) O(1) O(1)
Recurrent O(n · d2) O(n) O(n)
Convolutional O(k · n · d2) O(1) O(logk(n))
Self-Attention (restricted) O(r · n · d) O(1) O(n/r)
tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the
bottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel
as the embeddings, so that the two can be summed. There are many choices of positional encodings,
learned and ﬁxed [9].
In this work, we use sine and cosine functions of different frequencies:
P E(pos,2i) = sin(pos/100002i/dmodel )
P E(pos,2i+1) = cos(pos/100002i/dmodel )
where pos is the position and i is the dimension. That is, each dimension of the positional encoding
corresponds to a sinusoid. The wavelengths form a geometric progression from 2π to 10000 · 2π. We
chose this function because we hypothesized it would allow the model to easily learn to attend by
relative positions, since for any ﬁxed offset k, P Epos+k can be represented as a linear function of
P Epos.
We also experimented with using learned positional embeddings [9] instead, and found that the two
versions produced nearly identical results (see Table 3 row (E)). We chose the sinusoidal version
because it may allow the model to extrapolate to sequence lengths longer than the ones encountered
during training.
4 Why Self-Attention
In this section we compare various aspects of self-attention layers to the recurrent and convolu-
tional layers commonly used for mapping one variable-length sequence of symbol representations
(x1, ..., xn) to another sequence of equal length (z1, ..., zn), with xi, zi ∈ Rd, such as a hidden
layer in a typical sequence transduction encoder or decoder. Motivating our use of self-attention we
consider three desiderata.
One is the total computational complexity per layer. Another is the amount of computation that can
be parallelized, as measured by the minimum number of sequential operations required.
The third is the path length between long-range dependencies in the network. Learning long-range
dependencies is a key challenge in many sequence transduction tasks. One key factor affecting the
ability to learn such dependencies is the length of the paths forward and backward signals have to
traverse in the network. The shorter these paths between any combination of positions in the input
and output sequences, the easier it is to learn long-range dependencies [12]. Hence we also compare
the maximum path length between any two input and output positions in networks composed of the
different layer types.
As noted in Table 1, a self-attention layer connects all positions with a constant number of sequentially
executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of
computational complexity, self-attention layers are faster than recurrent layers when the sequence
length n is smaller than the representation dimensionality d, which is most often the case with
sentence representations used by state-of-the-art models in machine translations, such as word-piece
[38 ] and byte-pair [31 ] representations. To improve computational performance for tasks involving
very long sequences, self-attention could be restricted to considering only a neighborhood of size r in
6
the input sequence centered around the respective output position. This would increase the maximum
path length to O(n/r). We plan to investigate this approach further in future work.
A single convolutional layer with kernel width k < n does not connect all pairs of input and output
positions. Doing so requires a stack of O(n/k) convolutional layers in the case of contiguous kernels,
or O(logk(n)) in the case of dilated convolutions [ 18], increasing the length of the longest paths
between any two positions in the network. Convolutional layers are generally more expensive than
recurrent layers, by a factor of k. Separable convolutions [6 ], however, decrease the complexity
considerably, to O(k · n · d + n · d2). Even with k = n, however, the complexity of a separable
convolution is equal to the combination of a self-attention layer and a point-wise feed-forward layer,
the approach we take in our model.
As side beneﬁt, self-attention could yield more interpretable models. We inspect attention distributions
from our models and present and discuss examples in the appendix. Not only do individual attention
heads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic
and semantic structure of the sentences.
5 Training
This section describes the training regime for our models.
5.1 Training Data and Batching
We trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million
sentence pairs. Sentences were encoded using byte-pair encoding [3], which has a shared source-
target vocabulary of about 37000 tokens. For English-French, we used the signiﬁcantly larger WMT
2014 English-French dataset consisting of 36M sentences and split tokens into a 32000 word-piece
vocabulary [38 ]. Sentence pairs were batched together by approximate sequence length. Each training
batch contained a set of sentence pairs containing approximately 25000 source tokens and 25000
target tokens.
5.2 Hardware and Schedule
We trained our models on one machine with 8 NVIDIA P100 GPUs. For our base models using
the hyperparameters described throughout the paper, each training step took about 0.4 seconds. We
trained the base models for a total of 100,000 steps or 12 hours. For our big models,(described on the
bottom line of table 3), step time was 1.0 seconds. The big models were trained for 300,000 steps
(3.5 days).
5.3 Optimizer
We used the Adam optimizer [20] with β1 = 0.9, β2 = 0.98 and  = 10−9. We varied the learning
rate over the course of training, according to the formula:
lrate = d−0.5
model · min(step_num−0.5, step_num · warmup_steps−1.5) (3)
This corresponds to increasing the learning rate linearly for the ﬁrst warmup_steps training steps,
and decreasing it thereafter proportionally to the inverse square root of the step number. We used
warmup_steps = 4000.
5.4 Regularization
We employ three types of regularization during training:
Residual Dropout We apply dropout [33 ] to the output of each sub-layer, before it is added to the
sub-layer input and normalized. In addition, we apply dropout to the sums of the embeddings and the
positional encodings in both the encoder and decoder stacks. For the base model, we use a rate of
Pdrop = 0.1.
7
Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the
English-to-German and English-to-French newstest2014 tests at a fraction of the training cost.
Model BLEU Training Cost (FLOPs)
EN-DE EN-FR EN-DE EN-FR
ByteNet [18] 23.75
Deep-Att + PosUnk [39] 39.2 1.0 · 1020
GNMT + RL [38] 24.6 39.92 2.3 · 1019 1.4 · 1020
ConvS2S [9] 25.16 40.46 9.6 · 1018 1.5 · 1020
MoE [32] 26.03 40.56 2.0 · 1019 1.2 · 1020
Deep-Att + PosUnk Ensemble [39] 40.4 8.0 · 1020
GNMT + RL Ensemble [38] 26.30 41.16 1.8 · 1020 1.1 · 1021
ConvS2S Ensemble [9] 26.36 41.29 7.7 · 1019 1.2 · 1021
Transformer (base model) 27.3 38.1 3.3 · 1018
Transformer (big) 28.4 41.8 2.3 · 1019
Label Smoothing During training, we employed label smoothing of value ls = 0.1 [ 36 ]. This
hurts perplexity, as the model learns to be more unsure, but improves accuracy and BLEU score.
6 Results
6.1 Machine Translation
On the WMT 2014 English-to-German translation task, the big transformer model (Transformer (big)
in Table 2) outperforms the best previously reported models (including ensembles) by more than 2.0
BLEU, establishing a new state-of-the-art BLEU score of 28.4. The conﬁguration of this model is
listed in the bottom line of Table 3. Training took 3.5 days on 8 P100 GPUs. Even our base model
surpasses all previously published models and ensembles, at a fraction of the training cost of any of
the competitive models.
On the WMT 2014 English-to-French translation task, our big model achieves a BLEU score of 41.0,
outperforming all of the previously published single models, at less than 1/4 the training cost of the
previous state-of-the-art model. The Transformer (big) model trained for English-to-French used
dropout rate Pdrop = 0.1, instead of 0.3.
For the base models, we used a single model obtained by averaging the last 5 checkpoints, which
were written at 10-minute intervals. For the big models, we averaged the last 20 checkpoints. We
used beam search with a beam size of 4 and length penalty α = 0.6 [ 38 ]. These hyperparameters
were chosen after experimentation on the development set. We set the maximum output length during
inference to input length + 50, but terminate early when possible [38].
Table 2 summarizes our results and compares our translation quality and training costs to other model
architectures from the literature. We estimate the number of ﬂoating point operations used to train a
model by multiplying the training time, the number of GPUs used, and an estimate of the sustained
single-precision ﬂoating-point capacity of each GPU 5.
6.2 Model Variations
To evaluate the importance of different components of the Transformer, we varied our base model
in different ways, measuring the change in performance on English-to-German translation on the
development set, newstest2013. We used beam search as described in the previous section, but no
checkpoint averaging. We present these results in Table 3.
In Table 3 rows (A), we vary the number of attention heads and the attention key and value dimensions,
keeping the amount of computation constant, as described in Section 3.2.2. While single-head
attention is 0.9 BLEU worse than the best setting, quality also drops off with too many heads.
5We used values of 2.8, 3.7, 6.0 and 9.5 TFLOPS for K80, K40, M40 and P100, respectively.
8
Table 3: Variations on the Transformer architecture. Unlisted values are identical to those of the base
model. All metrics are on the English-to-German translation development set, newstest2013. Listed
perplexities are per-wordpiece, according to our byte-pair encoding, and should not be compared to
per-word perplexities.
N dmodel dff h dk dv Pdrop ls train PPL BLEU params
steps (dev) (dev) ×106
base 6 512 2048 8 64 64 0.1 0.1 100K 4.92 25.8 65
(A)
1 512 512 5.29 24.9
4 128 128 5.00 25.5
16 32 32 4.91 25.8
32 16 16 5.01 25.4
(B) 16 5.16 25.1 58
32 5.01 25.4 60
(C)
2 6.11 23.7 36
4 5.19 25.3 50
8 4.88 25.5 80
256 32 32 5.75 24.5 28
1024 128 128 4.66 26.0 168
1024 5.12 25.4 53
4096 4.75 26.2 90
(D)
0.0 5.77 24.6
0.2 4.95 25.5
0.0 4.67 25.3
0.2 5.47 25.7
(E) positional embedding instead of sinusoids 4.92 25.7
big 6 1024 4096 16 0.3 300K 4.33 26.4 213
Table 4: The Transformer generalizes well to English constituency parsing (Results are on Section 23
of WSJ)
Parser Training WSJ 23 F1
Vinyals & Kaiser el al. (2014) [37] WSJ only, discriminative 88.3
Petrov et al. (2006) [29] WSJ only, discriminative 90.4
Zhu et al. (2013) [40] WSJ only, discriminative 90.4
Dyer et al. (2016) [8] WSJ only, discriminative 91.7
Transformer (4 layers) WSJ only, discriminative 91.3
Zhu et al. (2013) [40] semi-supervised 91.3
Huang & Harper (2009) [14] semi-supervised 91.3
McClosky et al. (2006) [26] semi-supervised 92.1
Vinyals & Kaiser el al. (2014) [37] semi-supervised 92.1
Transformer (4 layers) semi-supervised 92.7
Luong et al. (2015) [23] multi-task 93.0
Dyer et al. (2016) [8] generative 93.3
In Table 3 rows (B), we observe that reducing the attention key size dk hurts model quality. This
suggests that determining compatibility is not easy and that a more sophisticated compatibility
function than dot product may be beneﬁcial. We further observe in rows (C) and (D) that, as expected,
bigger models are better, and dropout is very helpful in avoiding over-ﬁtting. In row (E) we replace our
sinusoidal positional encoding with learned positional embeddings [ 9], and observe nearly identical
results to the base model.
6.3 English Constituency Parsing
To evaluate if the Transformer can generalize to other tasks we performed experiments on English
constituency parsing. This task presents speciﬁc challenges: the output is subject to strong structural
9
constraints and is signiﬁcantly longer than the input. Furthermore, RNN sequence-to-sequence
models have not been able to attain state-of-the-art results in small-data regimes [37].
We trained a 4-layer transformer with dmodel = 1024 on the Wall Street Journal (WSJ) portion of the
Penn Treebank [ 25], about 40K training sentences. We also trained it in a semi-supervised setting,
using the larger high-conﬁdence and BerkleyParser corpora from with approximately 17M sentences
[37 ]. We used a vocabulary of 16K tokens for the WSJ only setting and a vocabulary of 32K tokens
for the semi-supervised setting.
We performed only a small number of experiments to select the dropout, both attention and residual
(section 5.4), learning rates and beam size on the Section 22 development set, all other parameters
remained unchanged from the English-to-German base translation model. During inference, we
increased the maximum output length to input length + 300. We used a beam size of 21 and α = 0.3
for both WSJ only and the semi-supervised setting.
Our results in Table 4 show that despite the lack of task-speciﬁc tuning our model performs sur-
prisingly well, yielding better results than all previously reported models with the exception of the
Recurrent Neural Network Grammar [8].
In contrast to RNN sequence-to-sequence models [37], the Transformer outperforms the Berkeley-
Parser [29] even when training only on the WSJ training set of 40K sentences.
7 Conclusion
In this work, we presented the Transformer, the ﬁrst sequence transduction model based entirely on
attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with
multi-headed self-attention.
For translation tasks, the Transformer can be trained signiﬁcantly faster than architectures based
on recurrent or convolutional layers. On both WMT 2014 English-to-German and WMT 2014
English-to-French translation tasks, we achieve a new state of the art. In the former task our best
model outperforms even all previously reported ensembles.
We are excited about the future of attention-based models and plan to apply them to other tasks. We
plan to extend the Transformer to problems involving input and output modalities other than text and
to investigate local, restricted attention mechanisms to efﬁciently handle large inputs and outputs
such as images, audio and video. Making generation less sequential is another research goals of ours.
The code we used to train and evaluate our models is available at https://github.com/
tensorflow/tensor2tensor.
Acknowledgements We are grateful to Nal Kalchbrenner and Stephan Gouws for their fruitful
comments, corrections and inspiration.
References
[1] Jimmy Lei Ba, Jamie Ryan Kiros, and Geoffrey E Hinton. Layer normalization. arXiv preprint
arXiv:1607.06450, 2016.
[2] Dzmitry Bahdanau, Kyunghyun Cho, and Yoshua Bengio. Neural machine translation by jointly
learning to align and translate. CoRR, abs/1409.0473, 2014.
[3] Denny Britz, Anna Goldie, Minh-Thang Luong, and Quoc V. Le. Massive exploration of neural
machine translation architectures. CoRR, abs/1703.03906, 2017.
[4] Jianpeng Cheng, Li Dong, and Mirella Lapata. Long short-term memory-networks for machine
reading. arXiv preprint arXiv:1601.06733, 2016.
[5] Kyunghyun Cho, Bart van Merrienboer, Caglar Gulcehre, Fethi Bougares, Holger Schwenk,
and Yoshua Bengio. Learning phrase representations using rnn encoder-decoder for statistical
machine translation. CoRR, abs/1406.1078, 2014.
[6] Francois Chollet. Xception: Deep learning with depthwise separable convolutions. arXiv
preprint arXiv:1610.02357, 2016.
10
[7] Junyoung Chung, Çaglar Gülçehre, Kyunghyun Cho, and Yoshua Bengio. Empirical evaluation
of gated recurrent neural networks on sequence modeling. CoRR, abs/1412.3555, 2014.
[8] Chris Dyer, Adhiguna Kuncoro, Miguel Ballesteros, and Noah A. Smith. Recurrent neural
network grammars. In Proc. of NAACL, 2016.
[9] Jonas Gehring, Michael Auli, David Grangier, Denis Yarats, and Yann N. Dauphin. Convolu-
tional sequence to sequence learning. arXiv preprint arXiv:1705.03122v2, 2017.
[10] Alex Graves. Generating sequences with recurrent neural networks. arXiv preprint
arXiv:1308.0850, 2013.
[11] Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun. Deep residual learning for im-
age recognition. In Proceedings of the IEEE Conference on Computer Vision and Pattern
Recognition, pages 770–778, 2016.
[12] Sepp Hochreiter, Yoshua Bengio, Paolo Frasconi, and Jürgen Schmidhuber. Gradient ﬂow in
recurrent nets: the difﬁculty of learning long-term dependencies, 2001.
[13] Sepp Hochreiter and Jürgen Schmidhuber. Long short-term memory. Neural computation,
9(8):1735–1780, 1997.
[14] Zhongqiang Huang and Mary Harper. Self-training PCFG grammars with latent annotations
across languages. In Proceedings of the 2009 Conference on Empirical Methods in Natural
Language Processing, pages 832–841. ACL, August 2009.
[15] Rafal Jozefowicz, Oriol Vinyals, Mike Schuster, Noam Shazeer, and Yonghui Wu. Exploring
the limits of language modeling. arXiv preprint arXiv:1602.02410, 2016.
[16] Łukasz Kaiser and Samy Bengio. Can active memory replace attention? In Advances in Neural
Information Processing Systems, (NIPS), 2016.
[17] Łukasz Kaiser and Ilya Sutskever. Neural GPUs learn algorithms. In International Conference
on Learning Representations (ICLR), 2016.
[18] Nal Kalchbrenner, Lasse Espeholt, Karen Simonyan, Aaron van den Oord, Alex Graves, and Ko-
ray Kavukcuoglu. Neural machine translation in linear time. arXiv preprint arXiv:1610.10099v2,
2017.
[19] Yoon Kim, Carl Denton, Luong Hoang, and Alexander M. Rush. Structured attention networks.
In International Conference on Learning Representations, 2017.
[20] Diederik Kingma and Jimmy Ba. Adam: A method for stochastic optimization. In ICLR, 2015.
[21] Oleksii Kuchaiev and Boris Ginsburg. Factorization tricks for LSTM networks. arXiv preprint
arXiv:1703.10722, 2017.
[22] Zhouhan Lin, Minwei Feng, Cicero Nogueira dos Santos, Mo Yu, Bing Xiang, Bowen
Zhou, and Yoshua Bengio. A structured self-attentive sentence embedding. arXiv preprint
arXiv:1703.03130, 2017.
[23] Minh-Thang Luong, Quoc V. Le, Ilya Sutskever, Oriol Vinyals, and Lukasz Kaiser. Multi-task
sequence to sequence learning. arXiv preprint arXiv:1511.06114, 2015.
[24] Minh-Thang Luong, Hieu Pham, and Christopher D Manning. Effective approaches to attention-
based neural machine translation. arXiv preprint arXiv:1508.04025, 2015.
[25] Mitchell P Marcus, Mary Ann Marcinkiewicz, and Beatrice Santorini. Building a large annotated
corpus of english: The penn treebank. Computational linguistics, 19(2):313–330, 1993.
[26] David McClosky, Eugene Charniak, and Mark Johnson. Effective self-training for parsing. In
Proceedings of the Human Language Technology Conference of the NAACL, Main Conference,
pages 152–159. ACL, June 2006.
11
[27] Ankur Parikh, Oscar Täckström, Dipanjan Das, and Jakob Uszkoreit. A decomposable attention
model. In Empirical Methods in Natural Language Processing, 2016.
[28] Romain Paulus, Caiming Xiong, and Richard Socher. A deep reinforced model for abstractive
summarization. arXiv preprint arXiv:1705.04304, 2017.
[29] Slav Petrov, Leon Barrett, Romain Thibaux, and Dan Klein. Learning accurate, compact,
and interpretable tree annotation. In Proceedings of the 21st International Conference on
Computational Linguistics and 44th Annual Meeting of the ACL, pages 433–440. ACL, July
2006.
[30] Oﬁr Press and Lior Wolf. Using the output embedding to improve language models. arXiv
preprint arXiv:1608.05859, 2016.
[31] Rico Sennrich, Barry Haddow, and Alexandra Birch. Neural machine translation of rare words
with subword units. arXiv preprint arXiv:1508.07909, 2015.
[32] Noam Shazeer, Azalia Mirhoseini, Krzysztof Maziarz, Andy Davis, Quoc Le, Geoffrey Hinton,
and Jeff Dean. Outrageously large neural networks: The sparsely-gated mixture-of-experts
layer. arXiv preprint arXiv:1701.06538, 2017.
[33] Nitish Srivastava, Geoffrey E Hinton, Alex Krizhevsky, Ilya Sutskever, and Ruslan Salakhutdi-
nov. Dropout: a simple way to prevent neural networks from overﬁtting. Journal of Machine
Learning Research, 15(1):1929–1958, 2014.
[34] Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, and Rob Fergus. End-to-end memory
networks. In C. Cortes, N. D. Lawrence, D. D. Lee, M. Sugiyama, and R. Garnett, editors,
Advances in Neural Information Processing Systems 28, pages 2440–2448. Curran Associates,
Inc., 2015.
[35] Ilya Sutskever, Oriol Vinyals, and Quoc VV Le. Sequence to sequence learning with neural
networks. In Advances in Neural Information Processing Systems, pages 3104–3112, 2014.
[36] Christian Szegedy, Vincent Vanhoucke, Sergey Ioffe, Jonathon Shlens, and Zbigniew Wojna.
Rethinking the inception architecture for computer vision. CoRR, abs/1512.00567, 2015.
[37] Vinyals & Kaiser, Koo, Petrov, Sutskever, and Hinton. Grammar as a foreign language. In
Advances in Neural Information Processing Systems, 2015.
[38] Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc V Le, Mohammad Norouzi, Wolfgang
Macherey, Maxim Krikun, Yuan Cao, Qin Gao, Klaus Macherey, et al. Google’s neural machine
translation system: Bridging the gap between human and machine translation. arXiv preprint
arXiv:1609.08144, 2016.
[39] Jie Zhou, Ying Cao, Xuguang Wang, Peng Li, and Wei Xu. Deep recurrent models with
fast-forward connections for neural machine translation. CoRR, abs/1606.04199, 2016.
[40] Muhua Zhu, Yue Zhang, Wenliang Chen, Min Zhang, and Jingbo Zhu. Fast and accurate
shift-reduce constituent parsing. In Proceedings of the 51st Annual Meeting of the ACL (Volume
1: Long Papers), pages 434–443. ACL, August 2013.
"""
query_embedding = get_embedding(query)

In [61]:
# calculate cosine similarity for test query via db
from sqlalchemy import select, desc, text
from sqlalchemy.orm import Session
from models import Domain, Topic, SubField

with Session(engine) as session:
    session.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
    statement = select(SubField, (1 - SubField.embedding.cosine_distance(query_embedding)).label('cosine_similarity')).order_by(desc('cosine_similarity')).limit(5)
    results = session.execute(statement).fetchall()
    
for result in results:
    print(f"{result.SubField.name}: {result.cosine_similarity}")

Computer Vision and Pattern Recognition: 0.20777135100706356
Linguistics and Language: 0.19101474609927516
Artificial Intelligence: 0.17084493686617297
Signal Processing: 0.162820484393691
Modeling and Simulation: 0.14683009423850946
